### Text

In [ ]:
Read
#read one file
input = sc.textFile("READ.md")

# read more file 讀取多個檔案,並且帶入目錄路徑
input = sc.textfile("~/Desktop")

#if file is small
input = sc.wholeTextFiles() #it will return a pair RDD,and the file name is k

In [ ]:
Save

#save the textFile
result.saveAsTextFile("")

### JSON

In [ ]:
#Read Json file
import json
input = sc.texFile("Json file")
data = input.map(lambda x: json.loads(x))

In [ ]:
#Save Json
data.filter(lambda x: 'lovesPandas' in x and x['lovesPandas']).map(lambda x:json.dumps(x)).saveAsTextFile("目錄")

### CSV

In [ ]:
#如果檔案沒有欄位有換行,textFile()
import csv
import StringIO

def loadRecord(line):
    """parse one row Csv record"""
    input = StringIO.StringIO(line)
    reader = csv.DictReader(input, filednames=["name", "favoriteAnimal"])
    return reader.next()

input = sc.textFile(inputCSVfile).map(loadRecord)

#如果檔案有欄位有換行
def loadRecords(fileNameContents):
    """parse whole file record"""
    input = StringIO.StringIO(fileNameContents[1])
    reader = csv.DictReader(input, filednames=["name", "favoriteAnimal"])
    return reader

fullFileData = sc.wholeTextFiles(inputFile).flatMap(loadRecords)

In [ ]:
#Save CSV file 建立一個映射保證輸出一致
import csv
import StringIO

def writeRecords(records):
    """output file"""
    output = StringIO.StringIO()
    writer = csv.DictWriter(output, filednames=["name", "favoriteAnimal"])
    for record in records:
        writer.writerow(record)
    return [output.getvalue()]

pandaLovers = input.filter(lambda x:x['favoriteAnimal']=='panda')
pandaLovers.mapPartitions(writeRecords).saveAsTextFile("outputfile")

### SequenceFiles

In [ ]:
#sc.sequence(path, keyClass, valueClass, minPartition) do the hadoop Writeabl class to see
#Read
data = sc.sequenceFile(inFile, "org.apache.io.Text", "org.apache.hadoop.io.IntWriteable")
#write


### Spark內建的輸入格式(textFile and SequenceFile )會自動處理某些壓縮格式,讀取壓縮檔時,壓縮編碼器會自動猜測壓縮的格式

### File System

### from local file system

### 必須這個檔案在所有節點上有相同的路徑

### HDFS

In [ ]:
hdfs://master:port/path

### 特過SparkSQL處理結構化資料

In [ ]:
1. copy hive-site.xml to Spark ./conf/ => build a HiveContext Object

In [ ]:
2. from pyspark.sql import HiveContext

hiveCtx = HiveContext(sc)
rows = hiveCtx.sql("Select * from users")
firstrow = rows.first()
print firstrow.name

In [ ]:
json = hiveCtx.jasonFile() => get row RDD

### SparkSQL => MySQL

In [ ]:
def createConnection() = {
    class.forName("com.mysql.jdbc.Driver").newInstance();
    DriverManager.getConnection("jdbc://localhost/test?user=jordan")
}

def extractValues(r: ResultSet) = {
    (r.getInt(1), r.getString(2))
}


data = new jdbcRDD(sc, createConnection, "select * from user where ?< = id and id <= ?",
                  lowerBound =1, upperBound=3, numPartition = 2, mapRow = extractValues)
print (data.collect().toList)